In [1283]:
import importlib
import os
import sys
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from dotenv import load_dotenv

In [1284]:
# Makes sure modules are available to the notebook that acts as an entrypoint

load_dotenv()

parent_dir = os.path.abspath('..')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [1285]:
# Manually reloading python module such that jupyter reflects changes without kernel restart

import plutus.code.calc.pcv2 as pc

importlib.reload(pc);

In [1286]:
import plutus.code.api_connector as ac

importlib.reload(ac);

In [1287]:
ticker = 'TSLA'
start_date = '2023-05-01-00:00'
end_date = '2023-05-02-00:00'
window_size = 180

api_connector = ac.ApiConnector()
api_connector.ticker = ticker
api_connector.start_date = start_date
api_connector.end_date = end_date

df = api_connector.request_daily_prices()

# df = df.iloc[:window_size]

Price for TSLA are available on disk. Reading


In [1288]:
pc_calculator = pc.PriceChannelCalculator()

# df = df.loc[0:390]

pc_calculator.dataframe = df
pc_calculator.window_size = window_size

pc_calculator.calculate_price_channels()

df.set_index('datetime', inplace=True)
df = df.loc['2023-05-01 09:30':'2023-05-01 16:00'] # 2, 3, 5, 11, 12, 15, 17, 22, 24
df.reset_index(inplace=True)

df

datetime     close     high       low     open  volume  \
0   2023-05-01 09:30:00  162.2000  163.240  162.1800  163.170  969515   
1   2023-05-01 09:31:00  161.6600  162.295  161.6000  162.200  762217   
2   2023-05-01 09:32:00  161.0300  161.720  161.0300  161.650  669457   
3   2023-05-01 09:33:00  161.1100  161.700  160.9400  161.040  725047   
4   2023-05-01 09:34:00  161.3999  161.500  160.9700  161.106  610605   
..                  ...       ...      ...       ...      ...     ...   
386 2023-05-01 15:56:00  161.9100  161.940  161.7100  161.710  237475   
387 2023-05-01 15:57:00  162.0350  162.040  161.8200  161.910  291482   
388 2023-05-01 15:58:00  161.9907  162.110  161.9100  162.040  272620   
389 2023-05-01 15:59:00  161.7800  162.005  161.7700  161.990  533335   
390 2023-05-01 16:00:00  161.8200  161.840  161.7703  161.790  524157   

        l_bound  
0    163.039502  
1    162.912046  
2    162.734890  
3    162.582085  
4    162.462737  
..          ...  
386  160.042674  
387  160.057342  
388  160.072221  
389  160.079189  
390  160.088324  

[391 rows x 7 columns]

In [1289]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(x=df.index, y=df['close'], line=dict(color="blue", width=1),name="Close"), row=1, col=1
)

# # Plot min extrema
# fig.add_trace(
#     go.Scatter(
#         x=pc_calculator.indices,
#         y=pc_calculator.minimas,
#         marker=dict(color='green', size=5),
#         mode='markers',
#         name='Min Extrema'
#     ),
#     row=1,
#     col=1
# )

# Plot min extrema trendline (lower channel bound)
# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df['lbf'],
#         line=dict(color="green", width=1),
#         name='LBF'
#     ),
#     row=1,
#     col=1
# )

# # Plot max extremas
# fig.add_trace(
#     go.Scatter(
#         x=pc_calculator.indices,
#         y=pc_calculator.maximas,
#         marker=dict(color='red', size=5),
#         mode='markers',
#         name='Max Extrema'
#     ),
#     row=1,
#     col=1
# )

# Plot max extrema trendline (upper channel bound)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['l_bound'],
        line=dict(color="red", width=1),
        name='Lower Bound'
    ),
    row=1,
    col=1
)

# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df['u_bound'],
#         line=dict(color="red", width=1),
#         name='Upper Bound'
#     ),
#     row=1,
#     col=1
# )

fig.update_layout(
    title={'text': f"{ticker}", 'x': 0.5},
    width=1000, height=500,
    xaxis={"rangeslider": {"visible": False}},
    plot_bgcolor="#EFF5F8"
)

fig.update_yaxes(visible=False, secondary_y=True)

In [1290]:
trace_value = 'lb_slopes'

x=df.index.values

y1=df['close'].values
y2=df[trace_value].values

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name='Close')

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis='y2')

# Create the layout with two y-axes
layout = go.Layout(
    title=f"{ticker} {df['datetime'].values[0]} - {df['datetime'].values[-1]}",
    yaxis=dict(),
    yaxis2=dict( overlaying='y', side='right'),
    height=650
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)
fig.update_yaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)

KeyError: 'lb_slopes'